In [1]:
import pandas as pd
from utils import *
pd.set_option('display.max_columns', None)

In [2]:
groups = {
    'ExCo': ['ceo', 'cto', 'cfo', 'coo', 'cmo', 'cio', 'cso', 'cpo', 'cco', 'cro', 'svp', 'evp', 'cdo', 'cbo', 'cxo'],
    'Founder': ['founder'],
    'Engineering': ['software engineer', 'data scientist'],
    'Leadership': ['president', 'managing director', 'director', 'vp', 'chairman', 'executive director', 'general manager', 'vp engineering'],
    'Board': ['board member', 'member board director', 'board directors', 'chairman board', 'advisory board member', 'board observer'],
    'Ownership': ['owner', 'partner', 'managing partner', 'founding partner'],
    'Investor':['investor'],
    'Advisory': ['advisor', 'consultant'],
    'Other': ['member', 'team member', 'associate', 'product manager', 'principal']
}

In [3]:
df_org         = pd.read_csv('../data/organizations.csv')
df_jobs        = pd.read_csv('../data/jobs.csv')
df_people      = pd.read_csv('../data/people.csv')

Cleaning the job dataframes

In [4]:
# Make a copy of the dataframe 'df_jobs' to avoid modifying the original dataframe
pple_new_roles = df_jobs.copy()

# Apply the 'clean_text' function to the 'title' column to clean the text in each title,
# and store the results in a new column called 'cleaned_title'
pple_new_roles['cleaned_title'] = pple_new_roles['title'].apply(lambda x: clean_text(x))

# Group the dataframe by the 'cleaned_title' column and count the occurrences of each unique title.
# Then, sort these counts in descending order.
sorted_counts = pple_new_roles.groupby('cleaned_title')['cleaned_title'].count().sort_values(ascending=False)

# Reset the index of the resulting Series to turn it into a DataFrame,
# and name the count column 'count'
sorted_counts_df = sorted_counts.reset_index(name='count')

In [6]:
sorted_counts_df.head(30)

,cleaned_title,count
0,founder,253796
1,ceo,163533
2,ceo founder,117598
3,founder ceo,79414
4,board member,63166
5,president,47706
6,director,39239
7,advisor,36934
8,managing director,33633
9,cto,32856


Given the number of chief executive jobs in the data, we want to use the same abreviation for each job description. We start by creating a chief_job_dataframe

In [7]:
# Find job titles containing the word 'chief' and reset the index
cumulative_chief = (
    find_job_word(sorted_counts_df, 'cleaned_title', 'chief')
    .reset_index()  # Reset the index to make it a column
    .assign(cumulative_count=lambda x: x['count'].cumsum())  # Add a new column with cumulative sum
)

# Normalize the cumulative count by the total cumulative count
norm = cumulative_chief['cumulative_count'].tail(1).values[0]
cumulative_chief['percentage_cumulative'] = cumulative_chief['cumulative_count'] / norm

# Apply various abbreviation functions to the cleaned titles
# We need to run abbreviation strategy three times as it is an incremental process.
# Check the abbreviated_cleaned_title, abbreviated_cleaned_title_round2, and abbreviated_cleaned_title_round3
# to observe the differences
cumulative_chief['abbreviated_cleaned_title'] = cumulative_chief['cleaned_title'].apply(abbreviate_title)
cumulative_chief['abbreviated_cleaned_title_round2'] = cumulative_chief['abbreviated_cleaned_title'].apply(abbreviate_title_round_2)
cumulative_chief['abbreviated_cleaned_title_round3'] = cumulative_chief['abbreviated_cleaned_title_round2'].apply(lambda x: remove_executives(x))

# Uncomment the following line to filter rows where percentage cumulative is less than 0.60
# cumulative_chief[cumulative_chief['percentage_cumulative'] < 0.60]

# Create a DataFrame for chief job titles with their counts, sorted in descending order
chief_job_dataframe = (
    find_job_word(sorted_counts_df, 'cleaned_title', 'chief')
    .groupby('cleaned_title')
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
    .rename(columns={'index': 'chief_jobs'})
)

# Apply abbreviation functions to the cleaned titles in the chief_df DataFrame
chief_df = find_job_word(sorted_counts_df, 'cleaned_title', 'chief')
chief_df['abbreviated_cleaned_title'] = chief_df['cleaned_title'].apply(abbreviate_title)
chief_df['abbreviated_cleaned_title_round2'] = chief_df['abbreviated_cleaned_title'].apply(abbreviate_title_round_2)
chief_df['abbreviated_cleaned_title_round3'] = chief_df['abbreviated_cleaned_title_round2'].apply(lambda x: remove_executives(x))

# Repeat the process to ensure cumulative_chief is updated with abbreviations and cumulative counts
cumulative_chief = (
    find_job_word(sorted_counts_df, 'cleaned_title', 'chief')
    .reset_index()  # Reset the index to make it a column
    .assign(cumulative_count=lambda x: x['count'].cumsum())  # Add a new column with cumulative sum
)
norm = cumulative_chief['cumulative_count'].tail(1).values[0]
cumulative_chief['percentage_cumulative'] = cumulative_chief['cumulative_count'] / norm
cumulative_chief['abbreviated_cleaned_title'] = cumulative_chief['cleaned_title'].apply(abbreviate_title)
cumulative_chief['abbreviated_cleaned_title_round2'] = cumulative_chief['abbreviated_cleaned_title'].apply(abbreviate_title_round_2)
cumulative_chief['abbreviated_cleaned_title_round3'] = cumulative_chief['abbreviated_cleaned_title_round2'].apply(lambda x: remove_executives(x))

# Uncomment the following line to filter rows where percentage cumulative is less than 0.60
# cumulative_chief[cumulative_chief['percentage_cumulative'] < 0.60]

# Apply abbreviation functions to the cleaned titles in the chief_df DataFrame
chief_df = find_job_word(sorted_counts_df, 'cleaned_title', 'chief')
chief_df['abbreviated_cleaned_title'] = chief_df['cleaned_title'].apply(abbreviate_title)
chief_df['abbreviated_cleaned_title_round2'] = chief_df['abbreviated_cleaned_title'].apply(abbreviate_title_round_2)
chief_df['abbreviated_cleaned_title_round3'] = chief_df['abbreviated_cleaned_title_round2'].apply(lambda x: remove_executives(x))

# Create a dictionary mapping cleaned_title to abbreviated_cleaned_title_round3
title_mapping = chief_df.set_index('cleaned_title')['abbreviated_cleaned_title_round3'].to_dict()

# Merge the new roles DataFrame with chief_df to get the abbreviated titles
new_df = pd.merge(pple_new_roles, chief_df[['cleaned_title', 'abbreviated_cleaned_title_round3']], 
                  how='left', 
                  on='cleaned_title')

# Create the final_title column based on the title mapping
new_df['final_title'] = new_df['cleaned_title'].where(
    ~new_df['cleaned_title'].isin(title_mapping),
    new_df['abbreviated_cleaned_title_round3']
)

# Drop rows where 'started_on' is NaN
new_df = new_df.dropna(subset=['started_on'])

# Move the word 'Founder' to the end of the title
new_df['final_final_title'] = new_df['final_title'].apply(move_founder_to_end)

# Group by the final_final_title and count occurrences, then sort and rename columns
tmp_df = new_df \
    .groupby('final_final_title') \
    .size() \
    .reset_index(name='count') \
    .sort_values(by='count', ascending=False) \
    .rename(columns={'index': 'three_letter_title'}) \
    .head(60)

# Calculate cumulative counts for tmp_df
cumulative_tmp_df = (
    tmp_df
    .reset_index()  # Reset the index to make it a column
    .assign(cumulative_count=lambda x: x['count'].cumsum())  # Add a new column with cumulative sum
)
norm = cumulative_tmp_df['cumulative_count'].tail(1).values[0]
cumulative_tmp_df['percentage_cumulative'] = cumulative_tmp_df['cumulative_count'] / norm

# Uncomment the following line to display the top 10 rows of cumulative_tmp_df
# cumulative_tmp_df.head(10)

# Create a DataFrame for three-letter titles and their counts, sorted in descending order
three_letter_titles = (
    new_df[new_df['cleaned_title'].str.len() == 3]
    .groupby('cleaned_title')
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
    .rename(columns={'index': 'three_letter_title'})
)

# # Calculate cumulative counts for three_letter_titles
# cumulative_three_letter = (
#     three_letter_titles
#     .reset_index()  # Reset the index to make it a column
#     .assign(cumulative_count=lambda x: x['count'].cumsum())  # Add a new column with cumulative sum
# )

# norm = cumulative_three_letter['cumulative_count'].tail(1).values[0]
# cumulative_three_letter['percentage_cumulative'] = cumulative_three_letter['cumulative_count'] / norm

# Uncomment the following line to display the top 10 rows of cumulative_three_letter
# cumulative_three_letter.head(10)

# Call the group_roles function to group roles in new_df based on the 'final_final_title' column
grouped_df = group_roles(new_df, 'final_final_title', groups)
grouped_df['group'] = grouped_df['group'].apply(lambda x: x if isinstance(x, list) else [x])

# Select relevant columns and convert 'started_on' to datetime
df_jobs_cleaned = grouped_df[['started_on', 'org_uuid', 'group', 'person_uuid']]
df_jobs_cleaned['started_on'] = pd.to_datetime(df_jobs_cleaned['started_on'], errors='coerce')
df_jobs_cleaned = df_jobs_cleaned[df_jobs_cleaned['started_on'].notna()]


In [10]:
sorted_counts = new_df.groupby('final_final_title')['final_final_title'].count().sort_values(ascending=False)

# Reset the index of the resulting Series to turn it into a DataFrame,
# and name the count column 'count'
sorted_counts_df = sorted_counts.reset_index(name='count')

In [12]:
sorted_counts_df.head(30)

,final_final_title,count
0,ceo founder,149065
1,founder,147412
2,ceo,112048
3,board member,35390
4,cto,33775
5,cfo,31452
6,president,31185
7,coo,28614
8,cto founder,25930
9,managing director,22682


In [6]:
df_jobs_cleaned.to_csv('../data/jobs_cleaned.csv',index=False)

# To read correctly the dataframe use the following
# df = pd.read_csv('data_output/jobs_cleaned.csv',
#                  converters={'group': str_to_list})

In [ ]:
# # Select relevant columns from df_org and convert 'founded_on' to datetime
# df_org_foundation = df_org[['uuid', 'founded_on', 'country_code', 'city', 'total_funding_usd']]
# df_org_foundation['founded_on'] = pd.to_datetime(df_org_foundation['founded_on'], errors='coerce')
# df_org_foundation = df_org_foundation[df_org_foundation['founded_on'].notna()]
# df_org_foundation = df_org_foundation[df_org_foundation['country_code'].notna()]

# # Create a list of dimension labels from the keys of the groups dictionary
# dimension_labels = list(groups.keys())